In [ ]:
!pip install langchain langchain-neo4j langchain-openai langchain_community

In [ ]:
from langchain_neo4j import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from dotenv import load_dotenv

load_dotenv("/content/.env")

# 실제 인스턴스 정보를 입력합니다.
NEO4J_URI = "bolt://localhost"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

embedding = OpenAIEmbeddings()

# Neo4j 그래프 객체 생성
graph = Neo4jVector.from_existing_graph(
    embedding=embedding,
    node_label="__Entity__",
    text_node_properties=["description"],
    embedding_node_property="embedding",
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Neo4jGraph 객체 추가 생성 (Cypher 쿼리 실행용)
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

In [ ]:
def fetch_entity_context(entity_name):
    context = {"name": entity_name}
    try:
        # 텍스트 청크 가져오기
        chunk_query = """
        MATCH (e:__Entity__ {name: $entity_name})<-[:HAS_ENTITY]-(c:__Chunk__)
        RETURN c.text AS text
        """
        chunk_result = neo4j_graph.query(chunk_query, {"entity_name": entity_name})
        context["text_chunks"] = [r["text"] for r in chunk_result] if chunk_result else ["No text chunk available"]

        # 커뮤니티 보고서 가져오기
        community_query = """
        MATCH (e:__Entity__ {name: $entity_name})-[:IN_COMMUNITY]->(com:__Community__)
        RETURN com.full_content AS report
        """
        community_result = neo4j_graph.query(community_query, {"entity_name": entity_name})
        context["community_reports"] = [r["report"] for r in community_result] if community_result else ["No community report available"]

        # 관련 엔티티 가져오기
        related_query = """
        MATCH (e:__Entity__ {name: $entity_name})-[:RELATED]->(related:__Entity__)
        RETURN related.name AS name, related.description AS description
        """
        related_result = neo4j_graph.query(related_query, {"entity_name": entity_name})
        context["related_entities"] = (
            [{"name": r["name"], "description": r["description"]} for r in related_result]
            if related_result else []
        )
    except Exception as e:
        context["error"] = f"Error fetching context: {str(e)}"
    return context

In [ ]:
def create_structured_context(all_contexts, query):
    context_str = "## 질문과 관련된 엔티티 정보\n\n"
    context_str += "아래는 질문에 답변하는 데 유용한 엔티티들의 구조화된 정보입니다:\n\n"

    for i, ctx in enumerate(all_contexts, 1):
        context_str += f"### 엔티티 {i}: {ctx['name']}\n"
        context_str += f"- **설명**: {ctx['description']}\n"
        context_str += "- **텍스트 청크**:\n"
        for chunk in ctx['text_chunks']:
            context_str += f"  - {chunk}\n"
        context_str += "- **커뮤니티 보고서**:\n"
        for report in ctx['community_reports']:
            context_str += f"  - {report}\n"
        if ctx['related_entities']:
            context_str += "- **관련 엔티티**:\n"
            for rel in ctx['related_entities']:
                context_str += f"  - {rel['name']}: {rel['description']}\n"
        else:
            context_str += "- **관련 엔티티**: 없음\n"
        context_str += "\n"
    return context_str

In [ ]:
# LLM 설정 (예: GPT-4o)
llm = ChatOpenAI(model="gpt-4o")

# 리트리버 설정
retriever = graph.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
# 질문 설정
query = "마일당 순이익(NET INCOME PER MILE)을 어떻게 분석해야 하나요?"
results = retriever.get_relevant_documents(query)

# 모든 엔티티의 컨텍스트 수집
all_contexts = []
for result in results:
    entity_name = result.metadata.get("name", "Unknown")
    description = result.page_content
    context = fetch_entity_context(entity_name)
    context["name"] = entity_name
    context["description"] = description
    all_contexts.append(context)

context_str = create_structured_context(all_contexts, query)
prompt = f"아래 맥락에 기반해서, 주어진 질문에 한국어로 답하세요\n\n**질문**: {query}\n\n**맥락**:\n{context_str}"
response = llm.invoke(prompt)
print("Final Response:")
print(response.content)

Global Retriever

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [ ]:
MAP_SYSTEM_PROMPT = """
---역할---
제공된 컨텍스트를 참고하여 사용자의 질문에 답하는 어시스턴트입니다.

---목표---
주어진 컨텍스트가 질문을 답하기에 적절하다면 질문에 대한 답을 한 뒤, 답변의 중요도 점수를 기입하여 JSON 형식으로 생성하세요.
정보가 부족하면 "모르겠습니다"라고 답하세요.

각 포인트는 다음을 포함해야 합니다:
- 답변: 질문에 대한 답변
- 중요도 점수: 0~100 사이의 정수.

데이터 참조 예:
"예시 문장 [Data: Reports (2, 7, 64, 46, 34, +more)]"
(한 참조에 5개 이상의 id는 "+more"를 사용)

출력 예:
{{"Answer": "답변 [Data: Reports (보고서 id들)]", "score": 점수}}
"""


map_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", MAP_SYSTEM_PROMPT),
        ("human", "question: {question}\n\n context: {context}"),
    ]
)

map_chain = map_prompt | llm | StrOutputParser()


In [ ]:
REDUCE_SYSTEM_PROMPT = """
---역할---
맵 단계에서 처리된 여러 결과를 종합하여 사용자의 질문에 답하는 어시스턴트입니다.

---목표---
제공된 맵 단계 결과를 바탕으로, 질문에 대한 종합적인 답변을 마크다운 형식으로 작성하세요.
중요도 점수를 고려하여 핵심적인 결과 위주로 반영하며, 불필요한 세부 사항은 제외하세요.
핵심 포인트와 시사점을 포함하고, 정보가 부족한 경우 "모르겠습니다"라고 답하세요.

---맵 단계 결과---
{report_data}

데이터 참조 형식은 아래를 따르세요:
"예시 문장 [Data: Reports (2, 7, 34, 46, 64, +more)]"
(참조 ID가 5개 이상일 경우 "+more" 사용)

대상 응답 길이 및 형식: {response_type}
"""

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", REDUCE_SYSTEM_PROMPT),
        ("human", "{question}"),
    ]
)

reduce_chain = reduce_prompt | llm | StrOutputParser()


In [ ]:
response_type: str = "multiple paragraphs"


def global_retriever(query: str, level: int, response_type: str = response_type) -> str:
    community_data = graph.query(
        """
    MATCH (c:__Community__)
    WHERE c.level = $level
    RETURN c.full_content AS output
    """,
        params={"level": level},
    )
    intermediate_results = []
    for community in tqdm(community_data, desc="Processing communities"):
        intermediate_response = map_chain.invoke(
            {"question": query, "context": community["output"]}
        )
        intermediate_results.append(intermediate_response)
    final_response = reduce_chain.invoke(
        {
            "report_data": intermediate_results,
            "question": query,
            "response_type": response_type,
        }
    )
    return final_response

In [ ]:
from tqdm import tqdm
print(global_retriever("이 책의 주제가 뭐야?", 1))